In [1]:
pip install gensim

In [2]:
pip install scikit-learn

## Q1


In [3]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
uciml_sms_spam_collection_dataset_path = kagglehub.dataset_download('uciml/sms-spam-collection-dataset')

print('Data source import complete.')


Data source import complete.


In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sms-spam-collection-dataset/spam.csv


In [5]:
df = pd.read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv', encoding='latin1')
print(df.head())

     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  


In [6]:
col_with_nan = [features for features in df.columns if df[features].isnull().sum()>1]
print(col_with_nan)

for ele in col_with_nan:
  print(ele, np.round(df[ele].isnull().mean(), 4))

  print(df[df[ele].notnull()][ele].head())

['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']
Unnamed: 2 0.991
95                                           PO Box 5249
281     the person is definitely special for u..... B...
444     HOWU DOIN? FOUNDURSELF A JOBYET SAUSAGE?LOVE ...
671     wanted to say hi. HI!!!\" Stop? Send STOP to ...
710      this wont even start........ Datz confidence.."
Name: Unnamed: 2, dtype: object
Unnamed: 3 0.9978
95                          MK17 92H. 450Ppw 16"
281                             why to miss them
899                         MK17 92H. 450Ppw 16"
1038                                          GE
2170    U NO THECD ISV.IMPORTANT TOME 4 2MORO\""
Name: Unnamed: 3, dtype: object
Unnamed: 4 0.9989
281                         just Keep-in-touch\" gdeve.."
1038                                              GNT:-)"
2255     Never comfort me with a lie\" gud ni8 and swe...
3525                  CALL 2MWEN IM BK FRMCLOUD 9! J X\""
4668      one day these two will become FREINDS FOREVER!"
Name: Unnamed: 4, dt

In [7]:
print(df.columns)

Index(['v1', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')


In [8]:
df = df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1)

In [9]:
df = df.rename(columns={'v1':'Labels', 'v2':'Message'})
df

,Labels,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [10]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
def preProcess(text):
  text = text.lower()

  tokens = word_tokenize(text)

  tokens = [word for word in tokens if word.isalnum]

  stop_words = set(stopwords.words('english'))
  tokens = [word for word in tokens if word not in stop_words]
  return tokens

In [12]:
df['Message'] = df['Message'].apply(preProcess)

In [13]:
df

,Labels,Message
0,ham,"[go, jurong, point, ,, crazy, .., available, b..."
1,ham,"[ok, lar, ..., joking, wif, u, oni, ...]"
2,spam,"[free, entry, 2, wkly, comp, win, fa, cup, fin..."
3,ham,"[u, dun, say, early, hor, ..., u, c, already, ..."
4,ham,"[nah, n't, think, goes, usf, ,, lives, around,..."
...,...,...
5567,spam,"[2nd, time, tried, 2, contact, u., u, å£750, p..."
5568,ham,"[ì_, b, going, esplanade, fr, home, ?]"
5569,ham,"[pity, ,, *, mood, ., ..., suggestions, ?]"
5570,ham,"[guy, bitching, acted, like, 'd, interested, b..."


In [14]:
import gensim.downloader as api
path = api.load("word2vec-google-news-300", return_path=True)
print(path)

[==================================================] 100.0% 1662.8/1662.8MB downloaded
/root/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz


In [15]:
from gensim.models import KeyedVectors

w2v_model = KeyedVectors.load_word2vec_format(path, binary=True)

In [16]:
w2v_model = KeyedVectors.load_word2vec_format(path, binary=True, limit=20000)

In [17]:
def message_to_vector(tokens, model):
  word_vectors = [model[word] for word in tokens if word in model]

  if word_vectors:
    return np.mean(word_vectors, axis=0)

  else:
    return np.zeros(model.vector_size)

In [18]:
df['Vector'] = df['Message'].apply(lambda tokens: message_to_vector(tokens, w2v_model))

In [19]:
print(df)

     Labels                                            Message  \
0       ham  [go, jurong, point, ,, crazy, .., available, b...   
1       ham           [ok, lar, ..., joking, wif, u, oni, ...]   
2      spam  [free, entry, 2, wkly, comp, win, fa, cup, fin...   
3       ham  [u, dun, say, early, hor, ..., u, c, already, ...   
4       ham  [nah, n't, think, goes, usf, ,, lives, around,...   
...     ...                                                ...   
5567   spam  [2nd, time, tried, 2, contact, u., u, å£750, p...   
5568    ham             [ì_, b, going, esplanade, fr, home, ?]   
5569    ham         [pity, ,, *, mood, ., ..., suggestions, ?]   
5570    ham  [guy, bitching, acted, like, 'd, interested, b...   
5571    ham                              [rofl, ., true, name]   

                                                 Vector  
0     [0.0041337446, 0.03335849, 0.008877841, 0.1967...  
1     [-0.014979045, 0.07120768, 0.11094157, 0.09838...  
2     [-0.0039550783, -0.03858236

In [20]:
from sklearn.model_selection import train_test_split

X = np.stack(df['Vector'].values)
y = df['Labels'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
from sklearn.linear_model import LogisticRegression

# model = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LogisticRegression())])

model = LogisticRegression()

model.fit(X_train, y_train)

LogisticRegression()

In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_pred = model.predict(X_test)

print("Accuracy: ", accuracy_score(y_test, y_pred))
# print("Precision: ", precision_score(y_test, y_pred, pos_label='spam'))
# print("Recall: ", recall_score(y_test, y_pred, pos_label='spam'))
# print("F1-Score: ", f1_score(y_test, y_pred, pos_label='spam'))

Accuracy:  0.9309417040358744


In [23]:
def predict_message_class(model, w2v_model, message):
  tokens = preProcess(message)
  vector = message_to_vector(tokens, w2v_model)
  prediction = model.predict([vector])[0]
  return prediction

In [24]:
message = "Congratulations! You've won a free ticket to Bahamas."

predict_message_class(model, w2v_model, message)

'spam'

## Q2

In [25]:
import kagglehub
crowdflower_twitter_airline_sentiment_path = kagglehub.dataset_download('crowdflower/twitter-airline-sentiment')

print('Data source import complete.')

Data source import complete.


In [26]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/twitter-airline-sentiment/Tweets.csv
/kaggle/input/twitter-airline-sentiment/database.sqlite
/kaggle/input/sms-spam-collection-dataset/spam.csv


In [27]:
data = pd.read_csv('/kaggle/input/twitter-airline-sentiment/Tweets.csv')
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [28]:
data = data.drop(columns=['airline_sentiment_confidence', 'retweet_count', 'airline','tweet_id', 'airline_sentiment_gold', 'negativereason', 'negativereason_confidence', 'tweet_location', 'tweet_created', 'user_timezone', 'negativereason_gold', 'tweet_coord', 'name'])

In [30]:
pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.9/113.9 kB 6.3 MB/s eta 0:00:00


In [31]:
import re

In [32]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [33]:
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [34]:
class preProcess2:
  def __init__(self, remove_emojis=True):
      self.remove_emojis = remove_emojis
      self.lemmatizer = WordNetLemmatizer()
      self.emoji_pattern = re.compile("["u"\U0001F600-\U0001F64F"
                                      "]+", flags=re.UNICODE)

  def remove_emoji(self, text):
    return self.emoji_pattern.sub(r' ', text)

  def preprocess(self, text):
    text = text.lower()
    text = self.expand_contractions(text)
    text = self.remove_unwanted(text)
    if self.remove_emojis:
      text = self.remove_emoji(text)
    text = self.lemmatize(text)
    return text

  def remove_unwanted(self, text):
    text = re.sub("@[A-Za-z0-9_]+"," ", text)
    text = re.sub(r'http\S+', ' ', text)
    text = re.sub("#[A-Za-z0-9_]+","", text)
    text = re.sub("[^0-9A-Za-z ]", "" , text)
    text = text.replace('  ',"")
    return text.strip()

  def expand_contractions(self, text):
    return contractions.fix(text)

  def get_wordnet_pos(self, word):
    """Map NLTK POS tags to WordNet POS tags."""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

  def lemmatize(self, text):
    words = nltk.word_tokenize(text)
    return " ".join([self.lemmatizer.lemmatize(w, self.get_wordnet_pos(w)) for w in words])

In [35]:
import contractions
contractions.fix("you're happy now")

'you are happy now'

In [36]:
preProcess = preProcess2()

data['clean text'] = data['text'].apply(preProcess.preprocess)
print(data)

      airline_sentiment                                               text  \
0               neutral                @VirginAmerica What @dhepburn said.   
1              positive  @VirginAmerica plus you've added commercials t...   
2               neutral  @VirginAmerica I didn't today... Must mean I n...   
3              negative  @VirginAmerica it's really aggressive to blast...   
4              negative  @VirginAmerica and it's a really big bad thing...   
...                 ...                                                ...   
14635          positive  @AmericanAir thank you we got on a different f...   
14636          negative  @AmericanAir leaving over 20 minutes Late Flig...   
14637           neutral  @AmericanAir Please bring American Airlines to...   
14638          negative  @AmericanAir you have my money, you change my ...   
14639           neutral  @AmericanAir we have 8 ppl so we need 2 know h...   

                                              clean text  
0   

In [37]:
data['Vector'] = data['clean text'].apply(lambda tokens: message_to_vector(tokens, w2v_model))

In [38]:
data

,airline_sentiment,text,clean text,Vector
0,neutral,@VirginAmerica What @dhepburn said.,what say,"[-0.24450684, 0.124938965, -0.10003662, 0.1527..."
1,positive,@VirginAmerica plus you've added commercials t...,plus you have add commercial to the experience...,"[-0.14605552, 0.11073705, 0.0065179123, 0.1412..."
2,neutral,@VirginAmerica I didn't today... Must mean I n...,i do not today must mean i need to take anothe...,"[-0.17007107, 0.11428494, -0.039289687, 0.1593..."
3,negative,@VirginAmerica it's really aggressive to blast...,it be really aggressive to blast obnoxious ent...,"[-0.18035813, 0.11219559, -0.005633545, 0.1301..."
4,negative,@VirginAmerica and it's a really big bad thing...,and it be a really big bad thing about it,"[-0.19542049, 0.096393295, 0.004699707, 0.1519..."
...,...,...,...,...
14635,positive,@AmericanAir thank you we got on a different f...,thank you we get on a different flight to chicago,"[-0.19280109, 0.08517097, -0.0019244025, 0.151..."
14636,negative,@AmericanAir leaving over 20 minutes Late Flig...,leave over 20 minute late flight no warning or...,"[-0.16301322, 0.09533497, -0.016636625, 0.1255..."
14637,neutral,@AmericanAir Please bring American Airlines to...,please bring american airline to,"[-0.16590118, 0.09626007, -0.0087178545, 0.159..."
14638,negative,"@AmericanAir you have my money, you change my ...",you have my money you change my flight and do ...,"[-0.16369033, 0.09011617, 0.009994879, 0.14263..."


In [39]:
X2 = np.stack(data['Vector'].values)
y2 = data['airline_sentiment'].values

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

In [40]:
model2 = LogisticRegression()

model2.fit(X2_train, y2_train)

LogisticRegression()

In [41]:
y2_pred = model2.predict(X2_test)

print("Accuracy: ", accuracy_score(y2_test, y2_pred))

Accuracy:  0.6700819672131147


In [42]:
def predict_tweet_sentiment(model, glove_model, tweet):
  preprocessor = preProcess2()

  cleaned_tweet = preprocessor.preprocess(tweet)

  vector = message_to_vector(cleaned_tweet.split(), glove_model)

  prediction = model.predict([vector])[0]

  return prediction

In [43]:
import gensim
from sklearn.manifold import TSNE
import gensim.downloader as api

In [44]:
glove_model = api.load('glove-wiki-gigaword-300')

[==================================================] 100.0% 376.1/376.1MB downloaded


In [45]:
tweet = "I can't believe how amazing the flight was! The crew was super friendly ✈️😊"

predict_tweet_sentiment(model2, glove_model, tweet)

'negative'